#Final Project

Context
You are a conservation biologist in charge of managing habitats for a protected
reserve. Your reserve includes several distinct patches of intact forest that harbor
populations of the endangered Warbling Babbler. You are worried about the longterm
fate of these populations if the habitats remain disconnected with limited
movement of individuals between them (i.e., limited migration between the
populations), so youʼd like to use funds from your budget to establish habitat
corridors that will allow individuals to move between the patches. However, you also
know that the populations have some phenotypic differences (like color) and you are
wondering how the frequencies of the phenotypes, and the overall population sizes,
will change in the populations once they are connected.


In [ ]:
# Import necessary modules
import matplotlib.pyplot as plt
import random

Your framework will include three types of classes: an Individual class, a Population class, and a Landscape class. Individuals exist in populations, and populations exist in a landscape. In each time step of your simulation, individuals can stay in the population where they started, or move to a
new population. The probabilities that individuals stay or leave are stored in a table called a dispersal matrix (more detail below).

In [ ]:
class Individual:
    """Class to hold information on indiviuals."""# Add a docstring
        
    def __init__(self, id, phenotype):
        """The constructor for the individual class."""
        self.id = id
        self.phenotype = phenotype

    def getPhenotype(self):
        return self.phenotype

    def setPhenotype(self, newPhenotype):
        self.phenotype = newPhenotype

Population constructors should create a new list for individuals in that population,
and Populations should have methods to add and remove individuals, as well as to
calculate and print the frequency of phenotypes among individuals.

In [ ]:
class Population:
    """Class to hold information on populations that is composed of individuals."""
    
    def __init__(self, id, popSize, phenotype): 
        """"The constructor for the population class."""
        self.id = id
        self.individuals = []
        for i in range(popSize):
            self.individuals.append(Individual("%s-%d" % (id, i+1), phenotype))

    def addIndividual(self, individual):
        self.individuals.append(individual)
    
    def removeIndividual(self, individual):
        self.individuals.remove(individual)
    
    def calculateFreq(self, phenotype):
        freq = 0
        for ind in self.individuals:
             if ind.phenotype == phenotype:
                 freq = freq + 1
        return freq
    
    def printFreq(self, phenotype):
        print("The %s phenotype frequency is %d" % (phenotype, self.calculateFreq(phenotype)))

The dispersal matrix consists of
the probabilities that an individual disperses between populations, or stays in place,
each time step. For Warbling Babblers, these probabilities correspond to their
probability of moving in a week (Fig. 1). If an individual moves from one population
to another, you should remove it from the list of individuals in the population where
it started, and add it to the list of individuals in the population where it went. There
are multiple ways to use the probabilities in the dispersal matrix to decide if an
individual moves in each time step, but one function you may find helpful is
random.choices(…) from the random module.

Landscape constructors should create a new list for the populations in that landscape, and Landscapes should have a method that uses the probabilities in the dispersal matrix to determine if an individual moves or stays each time step. Either the Population or Landscape constructors should set the starting population sizes. The total number of individuals across all populations should stay constant, but individual populations may change size.

In [ ]:
class Landscape:
    """Class to hold information on landscapes, which are composed of different populations."""
    
    def __init__(self,mapSize=10,popSize=10): # Add default values.
        """"The constructor for the population class, taking mapSize and popSize as arguments."""
        self.individuals=[] #creates an empty list for the individuals
        for i in range(popSize): #uses the established population size to run through the loop
            self.individuals.append(individual('LSU_%d' % (i+1))) #appends each individual to the list, updating the individual's ID accordingly. Uses i+1 so that the ID starts from 1 rather than 0.
        self.mapSize= mapSize #makes mapsize an attribute
        # Add a docstring
        # INSTRUCTIONS: This constructor should create a list of individuals in the population. The 
        # number of individuals should be equal to popSize and each individual should have
        # a unique id. The constructor should also record the mapSize as an attribute.